Load Dataset

In [ ]:
import pandas as pd
from rapidfuzz import process
import matplotlib.pyplot as plt
from IPython.display import display
df = pd.read_csv("dataset.csv", sep=";")

Inspect dataset

In [ ]:

# print number of column and lines
df.shape

In [ ]:
#preview first 5 rows of the dataset
print(df.head())

In [ ]:
#see column names, data types, and missing values
df.info()

Handle missing values

In [ ]:

#detect missing value by column  
df.isna().sum()

Load and inspect data

In [ ]:
# delete columns with all values missing 

df = df.drop('Cancellation comments', axis=1)
df = df.drop('Departure delay comments', axis=1)
df = df.drop('Arrival delay comments', axis=1)
df.shape[1]

In [ ]:
#detect mode value for column with missing values

df['Date'] = df['Date'].ffill()
df = df.dropna(subset=["Departure station"])  
df = df.dropna(subset=["Arrival station"])
for col in df:
    if df[col].dtype == float:
        float_modal = df[col].mean()
        df[col] = df[col].fillna(float_modal)
df.isna().sum()

In [ ]:
#convert Number of scheduled trains float values to integer
index = 0
result = 0
for value in df["Number of scheduled trains"]:
    if value - int(value) != 0:
        result = round(value)
        df.iloc[index, 5] = result
    index += 1

#convert Number of cancelled trains float values to integer
index = 0
result = 0
for value in df["Number of cancelled trains"]:
    if value - int(value) != 0:
        result = round(value)
        df.iloc[index,6] = result
    index += 1

#convert Number of trains delayed at departure float values to integer
index = 0
result = 0
for value in df["Number of trains delayed at departure"]:
    if value - int(value) != 0:
        result = round(value)
        df.iloc[index, 7] = result
    index += 1

#convert Number of trains delayed > 15min float values to integer
index = 0
result = 0
for value in df["Number of trains delayed > 15min"]:
    if value - int(value) != 0:
        result = round(value)
        df.iloc[index, 10] = result
    index += 1

#convert Number of trains delayed > 30min float values to integer
index = 0
result = 0
for value in df["Average delay of trains > 15min (if competing with flights)"]:
    if value - int(value) != 0:
        result = round(value)
        df.iloc[index, 12] = result
    index += 1

#convert Number of trains delayed > 60min float values to integer
index = 0
result = 0
for value in df["Number of trains delayed > 60min"]:
    if value - int(value) != 0:
        result = round(value)
        df.iloc[index, 13] = result
    index += 1

In [ ]:
#delete duplicates value 
df.drop_duplicates(inplace=True)
df.isna().sum()

Convert columns to appropriate data types.

In [ ]:
#correct dates bad formed with rapid fuzz
i = 0
m = 0
with open("utils/yyyy_mm.txt", 'r') as f:
    correct_dates = f.read().splitlines()
for bad_date in df['Date']:
     date, percent, _ = process.extractOne(bad_date, correct_dates)
     #df.loc[i, 'Date'] = "abs"
     if percent >= 80:
          df.iloc[i, 0] = date
     else:
          df.drop(i, inplace=True)
     i += 1
df.head()
df.isna().sum()
#print(m)

In [ ]:
#correct arrival stations bad named
i = 0
indexes = []
with open("utils/city.txt", 'r') as f:
    correct_stations = f.read().splitlines()
#print(correct_stations)
for bad_station in df['Departure station']:
     temp = process.extractOne(bad_station, correct_stations)
     if temp:
        station, percent, _ = temp
     if temp and percent >= 80:
         df.iloc[i, 2] = station
     else:
         indexes.append(i)
         #df.drop(df.index[i], inplace=True)
     i += 1
df.drop(df.index[indexes], inplace=True)
i = 0
st_indexes = []
for bad_station in df['Arrival station']:
     temp = process.extractOne(bad_station, correct_stations)
     if temp:
        station, percent, _ = temp
     if percent >= 80 and temp:
          df.iloc[i, 3] = station
     else:
         st_indexes.append(i)
     i += 1
df.drop(df.index[st_indexes], inplace=True)
df.isna().sum()

In [ ]:
#delete line if service col is empty 

In [ ]:
#dictionary
pays_par_ville = {
    "AIX EN PROVENCE TGV": "France",
    "ANGERS SAINT LAUD": "France",
    "ANGOULEME": "France",
    "ANNECY": "France",
    "ARRAS": "France",
    "AVIGNON TGV": "France",
    "BELLEGARDE (AIN)": "France",
    "BESANCON FRANCHE COMTE TGV": "France",
    "BORDEAUX ST JEAN": "France",
    "BREST": "France",
    "CHAMBERY CHALLES LES EAUX": "France",
    "DIJON VILLE": "France",
    "DOUAI": "France",
    "DUNKERQUE": "France",
    "GRENOBLE": "France",
    "LA ROCHELLE VILLE": "France",
    "LAVAL": "France",
    "LE CREUSOT MONTCEAU MONTCHANIN": "France",
    "LE MANS": "France",
    "LILLE": "France",
    "LYON PART DIEU": "France",
    "MACON LOCHE": "France",
    "MARNE LA VALLEE": "France",
    "MARSEILLE ST CHARLES": "France",
    "METZ": "France",
    "MONTPELLIER": "France",
    "MULHOUSE VILLE": "France",
    "NANCY": "France",
    "NANTES": "France",
    "NICE VILLE": "France",
    "NIMES": "France",
    "PARIS EST": "France",
    "PARIS LYON": "France",
    "PARIS MONTPARNASSE": "France",
    "PARIS NORD": "France",
    "PARIS VAUGIRARD": "France",
    "PERPIGNAN": "France",
    "POITIERS": "France",
    "REIMS": "France",
    "RENNES": "France",
    "SAINT ETIENNE CHATEAUCREUX": "France",
    "ST MALO": "France",
    "ST PIERRE DES CORPS": "France",
    "STRASBOURG": "France",
    "TOULON": "France",
    "TOULOUSE MATABIAU": "France",
    "TOURCOING": "France",
    "TOURS": "France",
    "VALENCE ALIXAN TGV": "France",
    "VANNES": "France",

    # Espagne
    "BARCELONA": "Espagne",
    "MADRID": "Espagne",

    # Allemagne
    "FRANCFORT": "Allemagne",
    "STUTTGART": "Allemagne",

    # Suisse
    "GENEVE": "Suisse",
    "LAUSANNE": "Suisse",
    "ZURICH": "Suisse",

    # Italie
    "ITALIE": "Italie"
}

# check if the journey is national or international
def get_etat(departure, arrival):
    pays_depart = pays_par_ville.get(departure, "Inconnu")
    pays_arrivee = pays_par_ville.get(arrival, "Inconnu")
    
    if pays_depart == pays_arrivee:
        return "National"
    else:
        return "International"

# Apply the function on each line
df["Service"] = df.apply(
    lambda row: get_etat(row["Departure station"], row["Arrival station"]), 
    axis=1
) 

df.head(10)


In [ ]:
##Generate summary statistics
print("____numerics statistics___")
print(df.describe())

In [ ]:
print("___categorials statistics___")
print(df.describe(include=['object']))

In [ ]:
print(df.columns.tolist())
df.columns = df.columns.str.strip()

In [ ]:
print("___Unique values per columns___")
print(df.nunique())

In [ ]:
##Statistiques descriptives

#List of all columns of delay
delay_columns = {
    'Average delay of late trains at departure',
    'Average delay of all trains at departure',
    'Average delay of all trains at arrival',
    'Average delay of late trains at arrival',
    'Average delay of trains > 15min (if competing with flights)'
}

#Let's do stats of column
stats = {}
for col in delay_columns:
    stats[col] = {
        "Moyenne": df[col].mean(),
        "Mediane": df[col].median(),
        "Ecart-type": df[col].std()
    }

#Display
print(stats)



In [ ]:
##Pourcentage de trains à l’heure, en retard, annulés.

#Name of column that we want to treat

cols = [
    "Number of scheduled trains",
    "Number of trains delayed at arrival",
    "Number of cancelled trains"
]

for c in cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

#Remove ligne where number is NaN or zero
df = df[df["Number of scheduled trains"] > 0].copy()

# Percentages
df["pct_delayed"] = df["Number of trains delayed at arrival"] / df["Number of scheduled trains"] * 100
df["pct_cancelled"] = df["Number of cancelled trains"] / df["Number of scheduled trains"] * 100
df["pct_on_time"] = 100 - (df["pct_delayed"] + df["pct_cancelled"])

# Average across the whole dataset
pct_summary = {
    "Average pct delayed": df["pct_delayed"].mean(),
    "Average pct cancelled": df["pct_cancelled"].mean(),
    "Average pct on time": df["pct_on_time"].mean()
}
display(pct_summary)

#Average by service
pct_by_service = df.groupby("Service").agg({
    "pct_delayed": "mean",
    "pct_cancelled": "mean",
    "pct_on_time": "mean"
})
display(pct_by_service)

#Average by date 
pct_by_date = df.groupby("Date").agg({
    "pct_delayed": "mean",
    "pct_cancelled": "mean",
    "pct_on_time": "mean"
})
display(pct_by_date)


In [ ]:
##Visualisations principales

#Histograms
delay_columns = [
    'Average delay of late trains at departure',
    'Average delay of all trains at departure',
    'Average delay of all trains at arrival',
    'Average delay of late trains at arrival',
    'Average delay of trains > 15min (if competing with flights)'
]

# Fig sizes 
plt.figure(figsize=(15, 10))
#Tab for colors: each fig with color
colors = ['green', 'royalblue', 'lime', 'salmon', 'magenta', 'goldenrod']

for i, col in enumerate(delay_columns, 1):
    plt.subplot(2, 3, i)
    plt.hist(df[col].dropna(), bins=30, color=colors[i], edgecolor='red', alpha=0.75)
    plt.title(f"Distribution: {col}", fontsize=10)
    plt.xlabel("Retard (minutes)")
    plt.ylabel("Nombre de trains")

plt.tight_layout()
#Display histograms
plt.show()

In [ ]:
##Boxplots

#Boxplot of late: station departure 
plt.figure(figsize=(12, 6))
df.boxplot(column='Average delay of all trains at departure', by='Departure station', grid=False)
plt.title("Retards moyens par gare de départ")
plt.suptitle("")
plt.xlabel("Gare de départ")
plt.ylabel("Retard (minutes)")
plt.xticks(rotation=45)
plt.show()

#Boxplot of late: station arrival
plt.figure(figsize=(12, 6))
df.boxplot(column='Average delay of all trains at arrival', by='Arrival station', grid=False)
plt.title("Retards moyens par gare d’arrivée")
plt.suptitle("")
plt.xlabel("Gare d’arrivée")
plt.ylabel("Retard (minutes)")
plt.xticks(rotation=45)
plt.show()



In [ ]:
##Courbes temporelles: late in fonction of month for the five type of late

#Find Month in Date 
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.to_period(freq = 'M')
df['Date'] = df['Date'].dt.to_period('M').astype(str)

# Group the data by month and calculate the average delay for each type of delay
month_statistics = df.groupby('Month')[
   [ 'Average delay of late trains at departure',
    'Average delay of all trains at departure',
    'Average delay of all trains at arrival',
    'Average delay of late trains at arrival',
    'Average delay of trains > 15min (if competing with flights)'
   ]
].mean()

#fig sizes
plt.figure(figsize=(14, 7))

for col in month_statistics.columns:
    plt.plot(month_statistics.index.astype(str), month_statistics[col], marker='o', label=col)
plt.title("Évolution mensuelle des retards")
plt.xlabel("Mois")
plt.ylabel("Retard moyen (minutes)")
plt.xticks(rotation=45)
plt.grid(True)
plt.legend(loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
##heatmaps to explore correlations between different variables

#Take column with numeric type
df_encoded = pd.get_dummies(df)
df_numeric = df_encoded.select_dtypes(include='number')

#Calculate correlation matrix who can be 1 or -1 or 0
# +1 → corrélation parfaite positive
#–1 → corrélation parfaite négative
# 0 → aucune corrélation


correlation_matrix = df_numeric.corr()
plt.figure(figsize=(12, 10))
plt.imshow(correlation_matrix, cmap='coolwarm', interpolation='none')
plt.colorbar(label='Coefficient de corrélation')

plt.xticks(range(len(correlation_matrix.columns)), correlation_matrix.columns, rotation=90)
plt.yticks(range(len(correlation_matrix.index)), correlation_matrix.index)

plt.title("Heatmap de corrélation : variables explicatives vs retards")
plt.tight_layout()
plt.show()

In [ ]:
##Insights

#Compare problematics station
arrival_delays = df.groupby('Arrival station')['Average delay of all trains at arrival'].mean().sort_values(ascending=False)
print(arrival_delays.head(10))
departure_delays = df.groupby('Departure station')['Average delay of all trains at departure'].mean().sort_values(ascending=False)
print(departure_delays.head(10))
##Graphs to display
arrival_delays.head(10).plot(kind='barh', color='yellow', title='Top gares en retard à l’arrivée')
plt.xlabel("Retard moyen (minutes)")
plt.show()
departure_delays.head(10).plot(kind='barh', color='pink', title='Top gares en retard au depart')
plt.xlabel("Retard moyen (minutes)")
plt.show()

In [ ]:
#Detect patterns;: Train delays tend to be more frequent during certain periods of the year.
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month
df['Date'] = df['Date'].dt.to_period('M').astype(str)

monthly_pattern = df.groupby('Month')['Average delay of all trains at arrival'].mean()

monthly_pattern.plot(marker='o', color='red', title='Retards moyens par mois')
plt.xlabel("Mois")
plt.ylabel("Retard moyen (minutes)")
plt.grid(True)
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()

In [ ]:
# write dataset
df.to_csv('cleaned_dataset.csv', index=False)

Perform feature engineering to create new useful variables